# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [2]:
#!pip install selenium

     ---------------------------------------- 6.9/6.9 MB 6.4 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 5.9 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [8]:
# Baixar o webdriver - ele recomenda:
# firefox - geckodriver
# chrome - chromedrive
#selenium roda em segundo plano e n tem q se preocupar com o carregamento
# Depois de baixado:
from selenium import webdriver

# Passo  1 - abrir o navegador
navegador = webdriver.Chrome()
    # Entrar em um site
navegador.get("https://www.google.com.br/")


In [7]:
# Passo 2 - Importar a base de dados
import pandas as pd

tabela = pd.read_excel("commodities.xlsx")
display(tabela)


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


In [15]:
# Passo 3 - Para cada produto da base de dados 
# Passo 4 - Pesquisar o preço do produto
# Passo 5 - Atualizar o preço na base de dados
    # Entrar em um link com o navegador já aberto com o código rodado anteriormente
    #link = "https://www.melhorcambio.com/milho-hoje"
    #navegador.get(link)
  # Selecionar um elemento dentro do site
  # find_element(foma de encontrar o elemento, código do elemento)
  # forma de enconmtrar é a mais usada o xpath
  # Para o código do elemento, tem que estar no site e inspencionar ele, clica no mouse com um quadrado no lado direito do canto 
  # esquerdo - clica no elemento - vai no códico clica com o botão direito, vai em copiar e depois copiar xpath
  # E por fim pegar o atributo com .get_attribute - olhar na inspeção qual atributo pegar, aqui queremos o valor então 
  # o atributo a ser pego é o value
    #preco_milho = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
# Fazer isso de forma mais automatizada:
# Percorrendo as linhas
for linha in tabela.index:
    # Localizar uma célula da tabela
    produto = tabela.loc[linha, "Produto"]
    
    # mudando os caracteres especiais para poder acessar o site
    produto = produto.replace("ó", "o").replace("ã", "a").replace("á", "a").replace("ç", "c").replace("ú", "u").replace("é", "e")
    
    # Entrar no link e pegar os valores dos diferentes produtos mudando só um nome no site - obs padrão
    link = f"https://www.melhorcambio.com/{produto}-hoje"
    navegador.get(link)
    preco = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    
    # Formatar os valores para ficar no padrão americano - melhorar os calculos
    preco = preco.replace(".", "").replace(",", ".")
   
    # Colocar o preço atual encontrado nos links na nossa base de dados e para garantir que é número passa um float
    tabela.loc[linha, "Preço Atual"] = float(preco)
    
display(tabela)

# outra forma de substituir os caracteres especiais: import unicodedata
# e produto = unicodedata.normalize("NFKD", produto).encode("ascii", "ignore")


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,NaN
1,Soja,163.59,162.64,NaN
2,Boi,282.20,276.05,NaN
3,Petróleo,424.37,391.29,NaN
4,Algodão,497.76,498.88,NaN
5,Açúcar,136.23,134.64,NaN
6,Café,1092.87,1087.87,NaN
7,Ouro,321.77,327.67,NaN
8,Trigo,1549.11,1546.08,NaN
9,Tilápia,9.05,9.05,NaN


In [16]:
# Passo 6 - Decidir quais produtos comprar
    # Vai preenche a tabela com true ou false
    # Mais eficiente do que fazer um for
tabela["Comprar"] = tabela["Preço Atual"] < tabela["Preço Ideal"]
display(tabela)


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,False
1,Soja,163.59,162.64,True
2,Boi,282.20,276.05,True
3,Petróleo,424.37,391.29,True
4,Algodão,497.76,498.88,False
5,Açúcar,136.23,134.64,True
6,Café,1092.87,1087.87,True
7,Ouro,321.77,327.67,False
8,Trigo,1549.11,1546.08,True
9,Tilápia,9.05,9.05,False


In [17]:
# Passo 7 - exportar a tabelo pro excel
    # Para tirar a colula de 0, 1, 2, 3, ... usa o index=false
tabela.to_excel("commodities_atualizado.xlsx", index = False)

# Sair do navegador
navegador.quit()
